# Getting things going and highlighting "the problem"

In [2]:
import pandas as pd
import sqlite3

ramphs = pd.read_csv("https://raw.githubusercontent.com/sfsheath/roman-amphitheaters/master/roman-amphitheaters.csv")
chronogrps = pd.read_csv("https://raw.githubusercontent.com/sfsheath/roman-amphitheaters/master/chronogrps.csv")

# I put an uncompressed version of pleiades-places into my folder on the github repo
pleiades = pd.read_csv("https://raw.githubusercontent.com/isaw-ga-3024/isaw-ga-3024.github.io/master/heath-sebastian/pleiades-places.csv")

# One note: because it takes a long time to load the pleiades data,
# avoid running this cell again unless necessary
# As I look back on class, perhaps this is why Mikael reported 
# that his replace was taking a long time. Was the data reloading across the internet?

In [3]:
# ramphs, chronogrps, and pleiades are now pandas DataFrames

print(type(ramphs))
print(type(chronogrps))
print(type(pleiades))

# Again, these are pandas DataFrames, not sql tables!

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [4]:
# OK, to our problem in class.

# inspect the values in both DataFrmes
print(ramphs['pleiades'].head(2))
print(pleiades['id'].head(2))

# They're different. Bummer.

0    http://pleiades.stoa.org/places/893989
1    http://pleiades.stoa.org/places/148217
Name: pleiades, dtype: object
0    265876
1    265877
Name: id, dtype: int64


# A tiny bit of string manipulation

In [5]:
tmp = "Hello World"
tmp.replace("Hello", "") # note that this doesn't remove the space

' World'

# A little bit of pandas dataframe manipulation

In [6]:
# first
ramphs.head(2)

,id,title,label,pleiades,type,capacity,modcountry,chronogrp,certainty,youtube,extmajor,extminor,arenamajor,arenaminor,latitude,longitude,elevation
0,duraEuroposAmphitheater,Amphitheater at Dura Europos,Dura,http://pleiades.stoa.org/places/893989,amphitheater,1000.0,Syria,Severan,NaN,NaN,50.0,44.0,31.0,25.0,34.749855,40.728926,223
1,arlesAmphitheater,Amphitheater at Arles,Arles,http://pleiades.stoa.org/places/148217,amphitheater,20000.0,France,Flavian,NaN,https://www.youtube.com/watch?v=oCz-76hb1LU,136.0,107.0,47.0,32.0,43.677778,4.631111,21


In [7]:
# then
ramphs['pleiades'].head(2)

# so it's easy to output either all columns, or just a selection.
# And it's likely that your notebook is trying to nicely format the output of ramphs.head(2).

0    http://pleiades.stoa.org/places/893989
1    http://pleiades.stoa.org/places/148217
Name: pleiades, dtype: object

In [8]:
# it's easy to create new columns
ramphs['new'] = "test"
ramphs['new'].head(5)

# And look!!!! Very important to note that pandas will automatically assign "test" to all rows.
# That's one of its great powers.

0    test
1    test
2    test
3    test
4    test
Name: new, dtype: object

In [9]:
# combining pandas and string manipulation
ramphs['new'].str.replace("e","").head(5) # do note the ".str" before ".replace"

# Again, pandas calculates result for all rows

0    tst
1    tst
2    tst
3    tst
4    tst
Name: new, dtype: object

In [10]:
# here's a useful construct, though dangerous in that 
# it changes data (a concern some of you raised in class)


ramphs['new'] = ramphs['new'].str.replace("t","")
ramphs['new'].head(5)

0    es
1    es
2    es
3    es
4    es
Name: new, dtype: object

# Review of adding tables to an sql database (to keep our terminology straight)

In [11]:
# The conn variable is a 'database connection'
# we are making this database in memory, meaning it won't be saved
conn = sqlite3.connect(':memory:')

# as convenience, I've put the above in a separate cell. You may not need to run it again.


In [12]:
# the following two lines create sql tables from pandas DataFrames
ramphs.to_sql('ramphs',conn,if_exists="replace")
chronogrps.to_sql('chronogrps', conn, if_exists="replace")
pleiades.to_sql('pleiades', conn, if_exists="replace")

# copy-paste, then edit one of the the above lines to make a pleiades table


In [13]:
pd.read_sql("""PRAGMA table_info(chronogrps)""",conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,chronogrp,TEXT,0,None,0
2,2,start,INTEGER,0,None,0
3,3,end,INTEGER,0,None,0


In [14]:
# let's test that we have sql tables
pd.read_sql("""SELECT id,pleiades FROM ramphs WHERE capacity > 40000""",conn)

# but that only confirms all is well working for the ramphs table.
# Good to test others as well.

,id,pleiades
0,romeFlavianAmphitheater,http://pleiades.stoa.org/places/423025


In [15]:
pd.read_sql("""SELECT chronogrp FROM chronogrps""",conn)

,chronogrp
0,Republican
1,Caesarean
2,Augustan
3,Julio-Claudian
4,Neronian
5,Flavian
6,First Century
7,Late1stEarly2nd
8,Hadrianic
9,Second Century


In [16]:
# a reminder
pd.read_sql("""PRAGMA table_info(pleiades)""",conn)
# will display the columns in the pleiades table (assuming you called it 'pleiades')
# look at those columns, which ones have 'chronological' information

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,authors,TEXT,0,None,0
2,2,bbox,TEXT,0,None,0
3,3,connectsWith,TEXT,0,None,0
4,4,created,TEXT,0,None,0
5,5,creators,TEXT,0,None,0
6,6,currentVersion,INTEGER,0,None,0
7,7,description,TEXT,0,None,0
8,8,extent,TEXT,0,None,0
9,9,featureTypes,TEXT,0,None,0


In [17]:
pleiades.head(2)

,authors,bbox,connectsWith,created,creators,currentVersion,description,extent,featureTypes,geoContext,...,path,reprLat,reprLatLong,reprLong,tags,timePeriods,timePeriodsKeys,timePeriodsRange,title,uid
0,"Spann, P., DARMC, R. Talbert, S. Gillies, R. W...","-3.606772, 39.460299, -3.606772, 39.460299",NaN,2010-09-24T19:02:22Z,P.O. Spann,14,"An ancient settlement, likely of Celtic origin...","{""type"": ""Point"", ""coordinates"": [-3.606772, 3...",settlement,Consuegra,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,"dare:ancient=1, dare:major=1, dare:feature=maj...",HRL,"hellenistic-republican,roman,late-antique","-330.0,640.0",Consabura/Consabrum,3fb26862377912da0f866fc310bcaf0c
1,"Spann, P., R. Warner, R. Talbert, S. Gillies, ...","-0.515639, 38.988847, -0.515639, 38.988847",NaN,2010-09-24T19:02:24Z,P.O. Spann,2,"An ancient place, cited: BAtlas 27 D3 Contestania","{""type"": ""Point"", ""coordinates"": [-0.515639, 3...",region,"SE coastal area, E Bastetania up to R. Júcar (...",...,/places/265877,38.988847,"38.988847,-0.515639",-0.515639,NaN,HR,"hellenistic-republican,roman","-330.0,300.0",Contestania,ff0b70ad7c02ce378753e5d02f00210c


In [18]:
pd.read_sql("""SELECT title,maxDate,minDate,timePeriods FROM pleiades LIMIT 10""",conn)

,title,maxDate,minDate,timePeriods
0,Consabura/Consabrum,640.0,-330.0,HRL
1,Contestania,300.0,-330.0,HR
2,Cueva de la Paloma,300.0,-330.0,HR
3,Dianium/Hemeroskopeion,2100.0,-750.0,ACHRLM
4,Ebura,300.0,-30.0,R
5,Ebusus,640.0,-750.0,ACHRL
6,Ebusus (island),2100.0,-750.0,ACHRLM
7,Edeba,300.0,-30.0,R
8,Edeta/Leiria,300.0,-550.0,CHR
9,Edetania,300.0,-330.0,HR


# More mixing of pandas and sql

In [19]:
# worth noting that the result of pd.read_sql statement is a pandas DataFrame
type(pd.read_sql("""SELECT id FROM ramphs WHERE capacity > 20000""",conn))

pandas.core.frame.DataFrame

In [20]:
# this means you can use pandas .groupby on it. This is familiar from last week
pd.read_sql("""SELECT modcountry,capacity FROM ramphs""",conn).groupby('modcountry').count()

,capacity
modcountry,
Albania,1
Algeria,6
Austria,3
Bulgaria,2
Croatia,2
Cyprus,0
France,18
Germany,2
Greece,1


In [22]:
# You now have all the piecces to move towards completing and going beyond
# the task I set in class. The task was:

# "For each amphitheater with a Pleiades ID,
# list that ID's ancient name(s) and chronological information
# First step... what are the steps?"

# At a minimum, write an SQL SELECT query that lists amphitheater IDs that have
# Pleiades identifiers along with the chronological information that pleiades records

# MORE INTERESTING AND HARDER: Write an SQL SELECT query that also lists the start column
# from the chronogrps table.
# So four columns at least: ramphs.id, chronogrps.start, pleiades.minDate, pleiades.maxDate

# once you do this it's easy to imagine a visualization of the difference between the founding
# of a city as indicated by pleiades and the construction of its amphitheater.
# A histogram of this would be nice. I'd calculate the difference using panda.
# (Or rather, that's how I did it.)

# Or think of some other manipulation that goes a step beyond the "at a minumum" query.


# Finally, as you ponder how to do the above, think about the conceptual range of DH as 
# found in this week's readings and as experienced while manipulating data in this
# ipython notebook. It's all DH and we love it all. How can theory and practice fit together?
# What other readings we've been assigned are relevant. This is just fodder for conversation.

pd.read_sql("""SELECT DISTINCT ramphs.id, pleiades.title, pleiades.minDate, pleiades.maxDate
FROM ramphs, chronogrps, pleiades LIMIT 10""", conn)

#need to figure out the join...

,id,title,minDate,maxDate
0,duraEuroposAmphitheater,Consabura/Consabrum,-330.0,640.0
1,duraEuroposAmphitheater,Contestania,-330.0,300.0
2,duraEuroposAmphitheater,Cueva de la Paloma,-330.0,300.0
3,duraEuroposAmphitheater,Dianium/Hemeroskopeion,-750.0,2100.0
4,duraEuroposAmphitheater,Ebura,-30.0,300.0
5,duraEuroposAmphitheater,Ebusus,-750.0,640.0
6,duraEuroposAmphitheater,Ebusus (island),-750.0,2100.0
7,duraEuroposAmphitheater,Edeba,-30.0,300.0
8,duraEuroposAmphitheater,Edeta/Leiria,-550.0,300.0
9,duraEuroposAmphitheater,Edetania,-330.0,300.0


In [31]:
pleiades['uri'] = "http://pleiades.stoa.org/places/"+pleiades['id'].astype(str)
pleiades['uri'].head(5)

#adding new column

0    http://pleiades.stoa.org/places/265876
1    http://pleiades.stoa.org/places/265877
2    http://pleiades.stoa.org/places/265878
3    http://pleiades.stoa.org/places/265880
4    http://pleiades.stoa.org/places/265882
Name: uri, dtype: object

In [28]:
pd.read_sql("""SELECT DISTINCT ramphs.id, pleiades.title, pleiades.minDate, pleiades.maxDate
FROM ramphs, chronogrps, pleiades WHERE ramphs.pleiades=pleiades.uri LIMIT 10""", conn)

#or am I???  I'm leaving my failed attempts to show my thought process, however incorrect it may be.

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 77))



DatabaseError: Execution failed on sql 'SELECT DISTINCT ramphs.id, pleiades.title, pleiades.minDate, pleiades.maxDate
FROM ramphs, chronogrps, pleiades WHERE ramphs.pleiades=pleiades.uri LIMIT 10': no such column: pleiades.uri

In [30]:
pd.read_sql("""PRAGMA table_info(pleiades)""",conn)

#no 'uri' column; curious...

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,authors,TEXT,0,None,0
2,2,bbox,TEXT,0,None,0
3,3,connectsWith,TEXT,0,None,0
4,4,created,TEXT,0,None,0
5,5,creators,TEXT,0,None,0
6,6,currentVersion,INTEGER,0,None,0
7,7,description,TEXT,0,None,0
8,8,extent,TEXT,0,None,0
9,9,featureTypes,TEXT,0,None,0


In [36]:
ramphs['pleiades'] = ramphs['pleiades'].str.replace("http://pleiades.stoa.org/places/","")
ramphs['pleiades'].head(5)
#I'm trying to provide a link between the two tables with 

0    893989
1    148217
2    167717
3    423025
4    423025
Name: pleiades, dtype: object

In [66]:
pleiades['id'].head(5)
#well, we have the same format...

0    265876
1    265877
2    265878
3    265880
4    265882
Name: id, dtype: int64

In [44]:
pd.read_sql("""SELECT ramphs.id, pleiades.id, pleiades.minDate, pleiades.maxDate
FROM ramphs, chronogrps, pleiades WHERE ramphs.pleiades = pleiades.id LIMIT 10""", conn)
#no results?

,id,title,minDate,maxDate


In [64]:
pd.read_sql("""SELECT ramphs.id, ramphs.modcountry FROM pleiades, ramphs
    WHERE pleiades.id = 893989 LIMIT 5""", conn)
#well this just seems wrong...

,id,modcountry
0,duraEuroposAmphitheater,Syria
1,arlesAmphitheater,France
2,lyonAmphitheater,France
3,ludusMagnusArena,Italy
4,romeFlavianAmphitheater,Italy


In [65]:
pd.read_sql("""SELECT ramphs.id, ramphs.modcountry FROM pleiades, ramphs
    WHERE pleiades.id = 423025 LIMIT 5""", conn)
#definitely wrong. different pleiades.id shouldn't be producing same results.

,id,modcountry
0,duraEuroposAmphitheater,Syria
1,arlesAmphitheater,France
2,lyonAmphitheater,France
3,ludusMagnusArena,Italy
4,romeFlavianAmphitheater,Italy


In [67]:
ramphs.to_sql('ramphs',conn,if_exists="replace")

# AHHH! Infuriating...missing such a simple step. Here is your head banging!

In [90]:
pd.read_sql("""SELECT DISTINCT ramphs.id, ramphs.title, pleiades.minDate, pleiades.maxDate
FROM ramphs, chronogrps, pleiades WHERE ramphs.pleiades = pleiades.id""", conn)

,id,title,minDate,maxDate
0,duraEuroposAmphitheater,Amphitheater at Dura Europos,-330.0,640.0
1,arlesAmphitheater,Amphitheater at Arles,-750.0,2100.0
2,lyonAmphitheater,Amphitheater at Lyon,-550.0,2100.0
3,ludusMagnusArena,Ludus Magnus Arena,-750.0,2100.0
4,romeFlavianAmphitheater,Flavian Amphitheater at Rome,-750.0,2100.0
5,romeAmphitheatrumCastrense,Amphitheatrum Castrense (Rome),-750.0,2100.0
6,eleutheropolisAmphitheater,Amphitheater at Eleutheropolis,-30.0,640.0
7,pompeiiAmphitheater,Amphitheater at Pompeii,-750.0,2100.0
8,meridaAmphitheater,Amphitheater at Mérida,-30.0,2100.0
9,newsteadAmphitheater,Amphitheater at Newstead,-30.0,300.0


In [91]:
pd.read_sql("""SELECT DISTINCT ramphs.id, ramphs.title, ramphs.modcountry, pleiades.minDate, pleiades.maxDate, chronogrps.start
    FROM ramphs, chronogrps, pleiades
    WHERE ramphs.pleiades = pleiades.id
    AND ramphs.chronogrp = chronogrps.chronogrp
    ORDER BY chronogrps.start""", conn)

#success!

,id,title,modcountry,minDate,maxDate,start
0,pompeiiAmphitheater,Amphitheater at Pompeii,Italy,-750.0,2100.0,-70
1,cumaeAmphitheater,Amphitheater at Cumae,Italy,-750.0,640.0,-70
2,pozzuoliEarlyAmphitheater,Early Amphitheater at Pozzuoli,Italy,-750.0,2100.0,-70
3,paestumAmphitheater,Amphitheater at Paestum,Italy,-750.0,2100.0,-70
4,avellaAmphitheater,Amphitheater at Abella,Italy,-330.0,640.0,-70
5,ferentoAmphitheater,Amphitheater at Ferento,Italy,-550.0,640.0,-70
6,sutriumAmphitheatre,Amphitheater at Sutrium,Italy,-550.0,640.0,-70
7,telesiaAmphitheatre,Amphitheater at Telesia,Italy,-330.0,640.0,-70
8,antiochAmphitheater,Amphitheater at Antioch,Turkey,-1750.0,2100.0,-70
9,beneventoAmphitheater,Amphitheater at Benevento,Italy,-330.0,2100.0,-70


# What if I don't accept the assignment?

In [ ]:
# That's OK. You can always use your own data or data you download. But you need to turn in
# an iPython notebook that demonstrates skills of approximately the same level of difficulty
# as what is above. At a minimum: a many-to-one or one-to-many relationship that is queried
# via SQL and then used to say something minimally interesting. The usual "go for it" applies.